# Generating the files needed for Track Hub

Here, you will find code and explanations on how we generated the needed files for [Track Hub](http://genome.ucsc.edu/cgi-bin/hgHubConnect).

All resulting files and folders are inside `track_hub/`, which contain the following elements:

 - `communities/`: Contains all the files that will be generated following the code in this notebook.
 - `bedToBigBed`: Program to convert .bed to bigBed format as explained here: https://genome.ucsc.edu/goldenPath/help/bigBed.html
 - `genomes.txt` / `hub.txt`: Files needed for Tracking Hub
 - `hg19.chrom.size`: File needed to execute this notebook, downloaded from https://genome.ucsc.edu/goldenPath/help/bigBed.html
 - `process_beds.sh`: Script that will convert all .bed files to bigBed format, deleting all .bed files. In practice, it executes Example \#2 from this link: https://genome.ucsc.edu/goldenPath/help/bigBed.html


Link provided to Track Hub: https://raw.githubusercontent.com/tjiagoM/gtex-transcriptome-modelling/master/track_hub/hub.txt

In [1]:
import pickle
import numpy as np

In [2]:
TISSUES = ['Adipose_Subcutaneous', 'Adipose_Visceral_Omentum', 'Adrenal_Gland', 'Artery_Aorta', 'Artery_Coronary',
           'Artery_Tibial', 'Brain_Amygdala', 'Brain_Anterior_cingulate', 'Brain_Caudate', 'Brain_Cerebellar',
           'Brain_Cerebellum', 'Brain_Cortex', 'Brain_Frontal_Cortex', 'Brain_Hippocampus', 'Brain_Hypothalamus',
           'Brain_Nucleus', 'Brain_Putamen', 'Brain_Spinal_cord', 'Brain_Substantia_nigra', 'Breast_Mammary_Tissue',
           'Cells_Cultured', 'Cells_EBV', 'Colon_Sigmoid', 'Colon_Transverse', 'Esophagus_Gastro', 'Esophagus_Mucosa',
           'Esophagus_Muscularis', 'Heart_Atrial', 'Heart_L_Vent', 'Kidney_Cortex', 'Liver', 'Lung', 'Minor_Salivary',
           'Muscle_Skeletal', 'Nerve_Tibial', 'Ovary', 'Pancreas', 'Pituitary', 'Prostate', 'Skin_Not_Sun_Epsd',
           'Skin_Sun_Epsd', 'Small_Intestine', 'Spleen', 'Stomach', 'Testis', 'Thyroid', 'Uterus', 'Vagina',
           'Whole_Blood']

In [3]:
# Saving the chromosome's limits, based on the file `hg19.chrom.sizes` downloaded from https://genome.ucsc.edu/goldenPath/help/bigBed.html

chr_limits = dict()
with open('track_hub/hg19.chrom.sizes', 'r') as reader:
    lines = reader.readlines()
    for line in lines:
        line_info = line.split('\t')
        chr_limits[line_info[0]] = int(line_info[1].strip())

In [4]:
# Saving start and end coordinates on all the known genes in gencode.v26.GRCh38.genes.gtf
dic_all_genes_info = dict()

with open('meta_data/gencode.v26.GRCh38.genes.gtf', 'r') as reader:
    lines = reader.readlines()
    for i, line in enumerate(lines):
        if i < 6: # before is just an header
            continue
        line_array = line.split('\t')
        chromosome = line_array[0]
        chr_start = int(line_array[3])
        chr_end = int(line_array[4])
        remain_info = line_array[8]
        gene_name = remain_info.split(';')[3].split(' ')[2][1:-1]
        
        # If the coordinates are over the known limits, set those coordinates to the known limits.
        if chr_end > chr_limits[chromosome]:
            chr_end = chr_limits[chromosome]
            if chr_start > chr_end:
                chr_start = chr_end
        
        dic_all_genes_info[gene_name] = {'chr': chromosome,
                                        'chr_start': chr_start,
                                        'chr_end': chr_end}

The following cell will generate all the needed `.bed` files (one for each community), as well as the `trackDb.txt` file needed for tracking Hub.

In [5]:
with open('track_hub/communities/trackDb.txt', 'w') as f_track_hubs:
    for tissue in TISSUES:
        try:
            for community_id in range(1, 999999):
                arr_com = []
                dic_community = pickle.load(open("svm_results/" + tissue + '_' + str(community_id) + ".pkl", "rb"))
                len_common = len(dic_community['genes'])

                with open(f'track_hub/communities/{tissue}_{community_id}.bed', 'w') as f:
                    for gene in dic_community['genes']:
                        if gene in dic_all_genes_info.keys():
                            gene_info = dic_all_genes_info[gene]
                            f.write(f'{gene_info["chr"]}\t{gene_info["chr_start"]}\t{gene_info["chr_end"]}\n')
                
                f_track_hubs.write(f'track {tissue}_{community_id}\n')
                f_track_hubs.write(f'bigDataUrl https://raw.githubusercontent.com/tjiagoM/gtex-transcriptome-modelling/master/track_hub/communities/{tissue}_{community_id}.bb\n')
                f_track_hubs.write(f'shortLabel {tissue}_{community_id}\n')
                f_track_hubs.write(f'longLabel {tissue}_{community_id}\n')
                f_track_hubs.write(f'type bigBed\n')
                f_track_hubs.write(f'\n')

        except Exception as e:
            pass

After all the cells from this notebook are executed, the next step is to execute the script `process_beds.sh` **inside the track_hub/ folder**  for final conversion needed for Tracking Hub.